In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import LongType, TimestampType, StringType

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 3, Finished, Available, Finished)

In [2]:
df = spark.sql("SELECT * FROM lakehouseSilver.clientes")
display(df)

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 81deade2-f458-4f1c-b310-2684db8466ae)

In [3]:
df_clientes = spark.sql("SELECT * FROM lakehouseSilver.clientes")

# 2. Cargar subdimensiones con cast y trim aplicado
dim_canal = spark.sql("""
    SELECT skCanal, CAST(TRIM(`Código`) AS STRING) AS codigoCanal
    FROM lakehouseSilver.dimensioncanalsilver
""")

dim_industria = spark.sql("""
    SELECT skIndustria, CAST(TRIM(`Código`) AS STRING) AS codigoSectorEconomico
    FROM lakehouseSilver.dimensionindustriasilver
""")

dim_segmento = spark.sql("""
    SELECT skSegmentoComercial, CAST(TRIM(`Código`) AS STRING) AS codigoSegmento
    FROM lakehouseSilver.dimensionsegmentocomercialsilver
""")

dim_valor = spark.sql("""
    SELECT skValorEstrategico, CAST(TRIM(`Código`) AS STRING) AS codigoValorEstrategico
    FROM lakehouseSilver.dimensionvalorestrategicosilver
""")

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 5, Finished, Available, Finished)

In [4]:
df_clientes = df_clientes \
    .withColumn("codigoCanal", F.trim(F.col("codigoCanal").cast("string"))) \
    .withColumn("codigoSectorEconomico", F.trim(F.col("codigoSectorEconomico").cast("string"))) \
    .withColumn("codigoSegmento", F.trim(F.col("codigoSegmento").cast("string"))) \
    .withColumn("codigoValorEstrategico", F.trim(F.col("codigoValorEstrategico").cast("string")))

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 6, Finished, Available, Finished)

In [5]:
df_final = df_clientes \
    .join(dim_canal, on="codigoCanal", how="left") \
    .join(dim_industria, on="codigoSectorEconomico", how="left") \
    .join(dim_segmento, on="codigoSegmento", how="left") \
    .join(dim_valor, on="codigoValorEstrategico", how="left")

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 7, Finished, Available, Finished)

In [6]:
try:
    df_sk_hist = spark.read.format("delta").table("lakehouseGold.dimcliente") \
        .select("codigoCliente", "skCliente")
    df_final = df_final.join(df_sk_hist, on="codigoCliente", how="left")
except:
    df_final = df_final.withColumn("skCliente", F.lit(None).cast("int"))

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 8, Finished, Available, Finished)

In [7]:
window_sk = Window.orderBy("codigoCliente")

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 9, Finished, Available, Finished)

In [8]:
df_final = df_final.withColumn("skCliente",
    F.when(F.col("skCliente").isNull(), F.row_number().over(window_sk) + 100000)
     .otherwise(F.col("skCliente"))
)

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 10, Finished, Available, Finished)

In [9]:
df_final = df_final \
    .withColumn("FechaCreacion", F.current_timestamp()) \
    .withColumn("EsActual", F.lit(True))

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 11, Finished, Available, Finished)

In [10]:
if df_final.groupBy("skCliente").count().filter("count > 1").count() > 0:
    raise Exception("❌ Error: claves sustitutas duplicadas en skCliente")

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 12, Finished, Available, Finished)

In [11]:
df_final = df_final.select(
    "skCliente", "codigoCliente", "nombreCliente", "fechaIngreso",
    "skSegmentoComercial", "skIndustria", "skValorEstrategico", "skCanal",
    "FechaCreacion", "EsActual"
)

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 13, Finished, Available, Finished)

In [12]:
display(df_final)


StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, acd3f9a6-9930-445d-9274-071ca0f09850)

In [14]:
df_final.write.mode("overwrite").format("delta").saveAsTable("lakehouseSilver.dimensionClienteSilver")

StatementMeta(, 73533424-fadc-4c8b-8eaa-9d5584545882, 16, Finished, Available, Finished)